In [ ]:
#Some Credits @campusX
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver # In production you ahve to use a persistent storage like Redis or Postgres

In [2]:
load_dotenv()

llm = ChatOpenAI()

In [16]:
class JokeState(TypedDict):

    topic_of_joke: str
    actual_joke: str
    explanation: str

In [18]:
def generate_joke(state: JokeState):

    prompt = f'generate a joke on the topic {state["topic_of_joke"]}'
    response = llm.invoke(prompt).content

    return {'actual_joke': response}

In [19]:
def generate_explanation(state: JokeState):

    prompt = f'write an explanation for the joke - {state["actual_joke"]}'
    response = llm.invoke(prompt).content

    return {'explanation': response}

In [21]:
graph = StateGraph(JokeState)

graph.add_node('generate_joke', generate_joke)
graph.add_node('generate_explanation', generate_explanation)

graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)

checkpointer1 = InMemorySaver()

workflow = graph.compile(checkpointer=checkpointer1)

In [24]:
config1 = {"configurable": {"thread_id": "1"}}
workflow.invoke({'topic_of_joke':'paid education'}, config=config1)

{'topic_of_joke': 'paid education',
 'actual_joke': 'Why did the student bring a ladder to paid education? To climb to the top of their class and reach their high-paying future!',
 'explanation': 'This joke plays on the double meaning of "climbing to the top" in the context of education and career success. By bringing a ladder to paid education, the student is humorously illustrating their determination to excel in school and eventually land a high-paying job. The image of physically climbing up a ladder mirrors the idea of moving up the academic and professional ranks to achieve success.'}

In [25]:
workflow.get_state(config1)

StateSnapshot(values={'topic_of_joke': 'paid education', 'actual_joke': 'Why did the student bring a ladder to paid education? To climb to the top of their class and reach their high-paying future!', 'explanation': 'This joke plays on the double meaning of "climbing to the top" in the context of education and career success. By bringing a ladder to paid education, the student is humorously illustrating their determination to excel in school and eventually land a high-paying job. The image of physically climbing up a ladder mirrors the idea of moving up the academic and professional ranks to achieve success.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f07e7ee-5a0d-6faa-8008-73c3d3425512'}}, metadata={'source': 'loop', 'step': 8, 'parents': {}, 'thread_id': '1'}, created_at='2025-08-21T11:06:33.198993+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f07e7ee-4424-6b57-8007-721ec948dc11'}}, 

In [26]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic_of_joke': 'paid education', 'actual_joke': 'Why did the student bring a ladder to paid education? To climb to the top of their class and reach their high-paying future!', 'explanation': 'This joke plays on the double meaning of "climbing to the top" in the context of education and career success. By bringing a ladder to paid education, the student is humorously illustrating their determination to excel in school and eventually land a high-paying job. The image of physically climbing up a ladder mirrors the idea of moving up the academic and professional ranks to achieve success.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f07e7ee-5a0d-6faa-8008-73c3d3425512'}}, metadata={'source': 'loop', 'step': 8, 'parents': {}, 'thread_id': '1'}, created_at='2025-08-21T11:06:33.198993+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f07e7ee-4424-6b57-8007-721ec948dc11'}},

In [ ]:
config2 = {"configurable": {"thread_id": "2"}}
workflow.invoke({'topic_of_joke':'unpaid_internships'}, config=config2)

{'topic_of_joke': 'unpaid internships',
 'actual_joke': 'Why did the unpaid intern bring a ladder to work? \nBecause they heard they had to climb the corporate ladder for a chance at a paid position!',
 'explanation': 'This joke is playing on the phrase "climbing the corporate ladder," which refers to advancing in a company\'s hierarchy or moving up in one\'s career. In this case, the unpaid intern brought a ladder to work because they took the phrase literally and thought they needed to physically climb a ladder to have a chance at a paid position. It highlights the struggle and competitiveness in the workplace, especially for interns or entry-level employees trying to secure a permanent, paid role.'}

In [29]:
workflow.get_state(config2)

StateSnapshot(values={'topic_of_joke': 'unpaid internships', 'actual_joke': 'Why did the unpaid intern bring a ladder to work? \nBecause they heard they had to climb the corporate ladder for a chance at a paid position!', 'explanation': 'This joke is playing on the phrase "climbing the corporate ladder," which refers to advancing in a company\'s hierarchy or moving up in one\'s career. In this case, the unpaid intern brought a ladder to work because they took the phrase literally and thought they needed to physically climb a ladder to have a chance at a paid position. It highlights the struggle and competitiveness in the workplace, especially for interns or entry-level employees trying to secure a permanent, paid role.'}, next=(), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f07e7f1-de0b-652c-8004-3fa3b60ba7c1'}}, metadata={'source': 'loop', 'step': 4, 'parents': {}, 'thread_id': '2'}, created_at='2025-08-21T11:08:07.569745+00:00', parent_config={'

In [30]:
list(workflow.get_state_history(config2))

[StateSnapshot(values={'topic_of_joke': 'unpaid internships', 'actual_joke': 'Why did the unpaid intern bring a ladder to work? \nBecause they heard they had to climb the corporate ladder for a chance at a paid position!', 'explanation': 'This joke is playing on the phrase "climbing the corporate ladder," which refers to advancing in a company\'s hierarchy or moving up in one\'s career. In this case, the unpaid intern brought a ladder to work because they took the phrase literally and thought they needed to physically climb a ladder to have a chance at a paid position. It highlights the struggle and competitiveness in the workplace, especially for interns or entry-level employees trying to secure a permanent, paid role.'}, next=(), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f07e7f1-de0b-652c-8004-3fa3b60ba7c1'}}, metadata={'source': 'loop', 'step': 4, 'parents': {}, 'thread_id': '2'}, created_at='2025-08-21T11:08:07.569745+00:00', parent_config={

### Time Travel

In [31]:
workflow.get_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f06cc6e-7232-6cb1-8000-f71609e6cec5"}})

StateSnapshot(values={}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_id': '1f06cc6e-7232-6cb1-8000-f71609e6cec5'}}, metadata=None, created_at=None, parent_config=None, tasks=(), interrupts=())

In [32]:
workflow.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f06cc6e-7232-6cb1-8000-f71609e6cec5"}})

EmptyInputError: Received no input for __start__

In [33]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic_of_joke': 'paid education', 'actual_joke': 'Why did the student bring a ladder to paid education? To climb to the top of their class and reach their high-paying future!', 'explanation': 'This joke plays on the double meaning of "climbing to the top" in the context of education and career success. By bringing a ladder to paid education, the student is humorously illustrating their determination to excel in school and eventually land a high-paying job. The image of physically climbing up a ladder mirrors the idea of moving up the academic and professional ranks to achieve success.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f07e7ee-5a0d-6faa-8008-73c3d3425512'}}, metadata={'source': 'loop', 'step': 8, 'parents': {}, 'thread_id': '1'}, created_at='2025-08-21T11:06:33.198993+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f07e7ee-4424-6b57-8007-721ec948dc11'}},

#### Updating State

In [34]:
workflow.update_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f06cc6e-7232-6cb1-8000-f71609e6cec5", "checkpoint_ns": ""}}, {'topic':'samosa'})

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f07e7f4-9ee9-6bfa-8000-7a30ac981df3'}}

In [35]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f07e7f4-9ee9-6bfa-8000-7a30ac981df3'}}, metadata={'source': 'update', 'step': 0, 'parents': {}, 'thread_id': '1'}, created_at='2025-08-21T11:09:21.480601+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06cc6e-7232-6cb1-8000-f71609e6cec5'}}, tasks=(PregelTask(id='0d6e0d3b-0f3c-4ae1-8654-e949524f0ca3', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'topic_of_joke': 'paid education', 'actual_joke': 'Why did the student bring a ladder to paid education? To climb to the top of their class and reach their high-paying future!', 'explanation': 'This joke plays on the double meaning of "climbing to the top" in the context of education and career success. By bringing a ladder to paid education, the student

In [36]:
workflow.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f06cc72-ca16-6359-8001-7eea05e07dd2"}})

EmptyInputError: Received no input for __start__

In [37]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f07e7f4-9ee9-6bfa-8000-7a30ac981df3'}}, metadata={'source': 'update', 'step': 0, 'parents': {}, 'thread_id': '1'}, created_at='2025-08-21T11:09:21.480601+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06cc6e-7232-6cb1-8000-f71609e6cec5'}}, tasks=(PregelTask(id='0d6e0d3b-0f3c-4ae1-8654-e949524f0ca3', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'topic_of_joke': 'paid education', 'actual_joke': 'Why did the student bring a ladder to paid education? To climb to the top of their class and reach their high-paying future!', 'explanation': 'This joke plays on the double meaning of "climbing to the top" in the context of education and career success. By bringing a ladder to paid education, the student

### Fault Tolerance

In [2]:
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import InMemorySaver
from typing import TypedDict
import time

In [3]:
# 1. Define the state
class CrashState(TypedDict):
    input: str
    step1: str
    step2: str

In [4]:
# 2. Define steps
def step_1(state: CrashState) -> CrashState:
    print("✅ Step 1 executed")
    return {"step1": "done", "input": state["input"]}

def step_2(state: CrashState) -> CrashState:
    print("⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)")
    time.sleep(1000)  # Simulate long-running hang
    return {"step2": "done"}

def step_3(state: CrashState) -> CrashState:
    print("✅ Step 3 executed")
    return {"done": True}

In [5]:
# 3. Build the graph
builder = StateGraph(CrashState)
builder.add_node("step_1", step_1)
builder.add_node("step_2", step_2)
builder.add_node("step_3", step_3)

builder.set_entry_point("step_1")
builder.add_edge("step_1", "step_2")
builder.add_edge("step_2", "step_3")
builder.add_edge("step_3", END)

checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)

In [ ]:
try:
    print("▶️ Running graph: Please manually interrupt during Step 2...")
    graph.invoke({"input": "start"}, config={"configurable": {"thread_id": 'thread-1'}})
except KeyboardInterrupt:
    print("❌ Kernel manually interrupted (crash simulated).")

▶️ Running graph: Please manually interrupt during Step 2...
✅ Step 1 executed
⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)


In [ ]:
# 6. Re-run to show fault-tolerant resume
print("\n🔁 Re-running the graph to demonstrate fault tolerance...")
final_state = graph.invoke(None, config={"configurable": {"thread_id": 'thread-1'}})
print("\n✅ Final State:", final_state)

In [ ]:
list(graph.get_state_history({"configurable": {"thread_id": 'thread-1'}}))